Eitan Zavorin - Module 6

In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances

In [2]:
players_df = pd.read_csv('Player.csv')
player_attributes_df = pd.read_csv('Player_Attributes.csv')

merged_df = pd.merge(players_df, player_attributes_df, on='player_api_id', how='inner')

merged_df = merged_df[merged_df['overall_rating'] >= 82]

merged_df

,id_x,player_api_id,player_name,player_fifa_api_id_x,birthday,height,weight,id_y,player_fifa_api_id_y,date,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
163,8,30895,Aaron Lennon,152747,1987-04-16 00:00:00,165.10,139,164,152747,2011-08-30 00:00:00,...,79.0,61.0,18.0,23.0,14.0,13.0,6.0,6.0,15.0,10.0
164,8,30895,Aaron Lennon,152747,1987-04-16 00:00:00,165.10,139,165,152747,2011-02-22 00:00:00,...,79.0,61.0,18.0,23.0,14.0,13.0,6.0,6.0,15.0,10.0
165,8,30895,Aaron Lennon,152747,1987-04-16 00:00:00,165.10,139,166,152747,2010-08-30 00:00:00,...,79.0,61.0,18.0,23.0,14.0,13.0,6.0,6.0,15.0,10.0
166,8,30895,Aaron Lennon,152747,1987-04-16 00:00:00,165.10,139,167,152747,2010-02-22 00:00:00,...,79.0,84.0,23.0,25.0,14.0,12.0,25.0,59.0,25.0,25.0
170,8,30895,Aaron Lennon,152747,1987-04-16 00:00:00,165.10,139,171,152747,2007-08-30 00:00:00,...,79.0,74.0,23.0,34.0,14.0,12.0,25.0,59.0,25.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183690,11057,35724,Zlatan Ibrahimovic,41236,1981-10-03 00:00:00,195.58,209,183691,41236,2008-02-22 00:00:00,...,80.0,72.0,24.0,21.0,28.0,12.0,24.0,36.0,24.0,24.0
183691,11057,35724,Zlatan Ibrahimovic,41236,1981-10-03 00:00:00,195.58,209,183692,41236,2007-08-30 00:00:00,...,80.0,72.0,24.0,21.0,28.0,12.0,24.0,36.0,24.0,24.0
183692,11057,35724,Zlatan Ibrahimovic,41236,1981-10-03 00:00:00,195.58,209,183693,41236,2007-02-22 00:00:00,...,80.0,72.0,15.0,21.0,28.0,12.0,6.0,36.0,8.0,6.0
183972,11075,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,183973,102359,2010-08-30 00:00:00,...,88.0,73.0,22.0,31.0,30.0,9.0,13.0,8.0,15.0,5.0


In [3]:
merged_df.drop_duplicates(subset=['player_name'], inplace=True)
merged_df = merged_df.dropna()
name_df = merged_df.set_index('player_name')

name_df

,id_x,player_api_id,player_fifa_api_id_x,birthday,height,weight,id_y,player_fifa_api_id_y,date,overall_rating,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
player_name,,,,,,,,,,,,,,,,,,,,,
Aaron Lennon,8,30895,152747,1987-04-16 00:00:00,165.10,139,164,152747,2011-08-30 00:00:00,82.0,...,79.0,61.0,18.0,23.0,14.0,13.0,6.0,6.0,15.0,10.0
Aaron Ramsey,15,75489,186561,1990-12-26 00:00:00,177.80,154,270,186561,2016-03-10 00:00:00,82.0,...,81.0,75.0,62.0,74.0,68.0,6.0,11.0,5.0,10.0,8.0
Abdulkader Keita,56,31012,157191,1981-08-06 00:00:00,182.88,172,900,157191,2012-08-31 00:00:00,82.0,...,70.0,84.0,13.0,28.0,23.0,6.0,7.0,8.0,7.0,15.0
Adam Johnson,109,24159,165740,1987-07-14 00:00:00,175.26,139,1744,165740,2011-08-30 00:00:00,82.0,...,80.0,65.0,38.0,32.0,33.0,8.0,5.0,13.0,13.0,9.0
Adil Rami,148,77741,183280,1985-12-27 00:00:00,190.50,194,2522,183280,2013-09-20 00:00:00,82.0,...,51.0,54.0,82.0,87.0,78.0,9.0,15.0,12.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Younes Belhanda,10968,179511,193738,1990-02-25 00:00:00,177.80,165,182361,193738,2012-08-31 00:00:00,82.0,...,86.0,70.0,49.0,54.0,52.0,15.0,16.0,11.0,6.0,16.0
Yuri Zhirkov,10996,40825,154950,1983-08-20 00:00:00,180.34,165,182785,154950,2010-02-22 00:00:00,83.0,...,82.0,69.0,83.0,77.0,84.0,8.0,20.0,73.0,20.0,20.0
Ze Roberto,11039,38843,28765,1974-07-06 00:00:00,172.72,159,183482,28765,2010-08-30 00:00:00,82.0,...,91.0,68.0,73.0,84.0,84.0,13.0,14.0,11.0,10.0,9.0


In [4]:
# drop non-quantitative and irrelevant columns
df = name_df.drop(columns=['overall_rating', 'potential', 'attacking_work_rate', 'defensive_work_rate', 'id_y', 'player_fifa_api_id_y', 'id_x', 'player_api_id', 'player_fifa_api_id_x', 'birthday', 'height', 'weight', 'preferred_foot', 'date', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes'])

# drop less significant play style columns
df = df.drop(columns=['jumping', 'balance', 'reactions', 'marking'])
df

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,...,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,standing_tackle,sliding_tackle
player_name,,,,,,,,,,,,,,,,,,,,,
Aaron Lennon,79.0,68.0,27.0,77.0,69.0,85.0,62.0,54.0,53.0,88.0,...,86.0,52.0,56.0,56.0,36.0,79.0,79.0,61.0,23.0,14.0
Aaron Ramsey,75.0,75.0,58.0,83.0,79.0,81.0,70.0,70.0,80.0,82.0,...,92.0,69.0,75.0,74.0,69.0,83.0,81.0,75.0,74.0,68.0
Abdulkader Keita,75.0,72.0,77.0,73.0,69.0,86.0,69.0,57.0,76.0,83.0,...,81.0,89.0,78.0,75.0,37.0,85.0,70.0,84.0,28.0,23.0
Adam Johnson,83.0,77.0,43.0,77.0,66.0,86.0,76.0,75.0,71.0,84.0,...,76.0,64.0,76.0,61.0,47.0,80.0,80.0,65.0,32.0,33.0
Adil Rami,52.0,47.0,87.0,65.0,63.0,53.0,43.0,61.0,76.0,69.0,...,66.0,90.0,52.0,78.0,83.0,28.0,51.0,54.0,87.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Younes Belhanda,75.0,76.0,67.0,84.0,69.0,83.0,78.0,78.0,75.0,84.0,...,80.0,69.0,77.0,78.0,66.0,82.0,86.0,70.0,54.0,52.0
Yuri Zhirkov,88.0,70.0,62.0,82.0,71.0,87.0,75.0,78.0,73.0,85.0,...,88.0,74.0,81.0,70.0,84.0,79.0,82.0,69.0,77.0,84.0
Ze Roberto,78.0,58.0,65.0,86.0,66.0,80.0,74.0,74.0,85.0,92.0,...,77.0,69.0,56.0,63.0,76.0,79.0,91.0,68.0,84.0,84.0


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
all_player_names = np.array(list(df.index.values))
training_names, testing_names = train_test_split(all_player_names, test_size=0.1)

In [7]:
training_names.shape, testing_names.shape

((549,), (61,))

In [8]:
k_nn = 10

In [15]:
prediction_rows = []

for test_player_name in testing_names:

    # This test player
    target_player_object = df.loc[test_player_name]
    
    # List of similar movies
    test_player_similarities = []

    #Generating distances from that player to all the others
    distances = scipy.spatial.distance.cdist(df, [target_player_object], metric="cosine")[:,0]

    query_distances = list(zip(df.index, distances))
    
    
    for similar_player_name, similar_attribute_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[1:11]:

        test_player_similarities.append(similar_player_name)

    
    # iterate through the top k movies and predict genre and rating
    relevant_ratings = []
    relevant_genre_count = {}
    for this_player_name in top_movies["movie"]:
        # add relevant ratings
        # TODO add relevant rating to `relevant_ratings`
        
        # Track genres
        # TODO add this similar movie's genre to map of genre counts
            
    # Get the most common genre from the neighbors
    predicted_genre = # TODO
    
    # Take the average rating of k neighbors for prediction
    predicted_rating = # TODO
    
    # Track predictions
    prediction_rows.append({
        "imdb_id": test_movie_id,
        "genre": predicted_genre,
        "rating": predicted_rating,
    })


In [ ]:
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances

In [ ]:
#Setting the player we will be comparing to
target_player_name = 'Wayne Rooney'

In [ ]:
#Gathering the attributes for that player
target_player_attributes = df.loc[target_player_name]


#Generating distances from that player to all the others
distances = scipy.spatial.distance.cdist(df, [target_player_attributes], metric="cosine")[:,0]

query_distances = list(zip(df.index, distances))